## Need Install Package paths

## Begin Tutorial 

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from pathlib import Path

import spatialdata as sd
import spatialdata_plot

import sopa

# optional: set the parallelization backend to use dask
sopa.settings.parallelization_backend = "dask"

## SpatialData (a tool to align and store spatial samples)

In [2]:
zarr_path = Path("/Users/rushin.gindra/Documents/Research/TeachingCourses/cell_seg/spatialdata-sandbox/xenium_2.0.0_pancreas_io/data.zarr")

In [ ]:
sdata = sd.read_zarr(zarr_path)

## Pancreas Xenium sample

### Different components:
A SpatialData object is designed to store and manage spatial biological data. It is composed of several key components that allow for comprehensive analysis and visualization of spatial datasets. Below are the main components of a SpatialData object:

- **Images (DataArrays)**
  - **Description:** The Images component stores the visual data, such as microscopy images or other spatially resolved imaging data.
  - **Purpose:** Provides the primary visual data that forms the basis of spatial analysis.
  - **Example:** High-resolution microscopy images of tissue sections.

- **Labels (DataArrays)**
  - **Description:** The Labels component contains categorical or numerical annotations assigned to specific regions or points within the images.
  - **Purpose:** Allows for the classification or categorization of spatial features.
  - **Example:** Cell type labels assigned to specific regions in an image.

- **Points (GeoDataFrame)**
  - **Description:** The Points component stores the coordinates of specific points of interest within the spatial data.
  - **Purpose:** Identifies precise locations within the images that are relevant for analysis.
  - **Example:** Coordinates of cell centroids or specific landmarks.

- **Shapes (GeoDataFrame)**
  - **Description:** The Shapes component stores geometric shapes that define regions of interest within the spatial data.
  - **Purpose:** Allows for the definition of complex regions or boundaries within the images.
  - **Example:** Polygon shapes outlining specific tissue regions.

- **Tables (AnnData)**
  - **Description:** The Tables component contains tabular data associated with the spatial data, such as metadata or feature measurements.
  - **Purpose:** Provides additional information or measurements linked to the spatial data.
  - **Example:** Gene expression data corresponding to specific cells or regions.

In [ ]:
sdata

### Basic plotting

In [ ]:
sdata.pl.render_images("he_image").pl.show(coordinate_systems='global', title="histology")

In [ ]:
sdata.pl.render_images("morphology_focus").pl.show(coordinate_systems='global', title="morphology")

In [ ]:
sdata.pl.render_labels('cell_labels').pl.show(coordinate_systems='global', title="cell_labels")

In [ ]:
sdata.pl.render_shapes('cell_circles').pl.show(coordinate_systems='global', title="cell_circles")

In [ ]:
sdata.pl.render_images('he_image')\
    .pl.render_shapes('cell_circles', outline_alpha=0.5, fill_alpha=0.5)\
        .pl.show(coordinate_systems='global', title="cell_circles_overlay")

In [ ]:
sdata.pl.render_images('morphology_focus')\
    .pl.render_shapes('cell_circles', outline_alpha=0.5, fill_alpha=0, outline_color='white')\
        .pl.show(coordinate_systems='global', title="cell_circles_dapi_overlay")

## Tissue Segmentation

In [ ]:
sopa.segmentation.tissue(sdata, image_key='he_image')

In [ ]:
sdata

In [ ]:
sdata.pl.render_images('he_image')\
    .pl.render_shapes('region_of_interest', outline_alpha=1.0, fill_alpha=0, outline_color='black')\
        .pl.show(coordinate_systems='global', title="Tissue Segmentation")

## Annotating with Napari

In [ ]:
from napari_spatialdata import Interactive

interactive = Interactive(sdata)
interactive.run()

In [ ]:
sdata

In [ ]:
sdata.pl.render_images('he_image')\
    .pl.render_shapes('annotations', outline_alpha=1.0, fill_alpha=0, outline_color='black')\
        .pl.show(coordinate_systems='global', title="Basic Annotation")

## ROI Crop (Optional)

In [ ]:
annotations = sdata.shapes["annotations"]
# annotations

# Step 1: Extract the polygon from the geodataframe
polygon = annotations.geometry.iloc[0]

# Step 2: Get the bounds of the polygon
min_x, min_y, max_x, max_y = polygon.bounds

# Step 3: Print the coordinates
print(f"Minimum X: {min_x}, Maximum X: {max_x}")
print(f"Minimum Y: {min_y}, Maximum Y: {max_y}")

In [ ]:
sdata_cropped = sdata.query.bounding_box(
    min_coordinate=[min_x, min_y], max_coordinate=[max_x, max_y], axes=("x", "y"), target_coordinate_system='global'
)

sdata_cropped

In [ ]:
sdata_cropped.pl.render_images('he_image', alpha=1.0)\
    .pl.render_shapes('cell_boundaries', outline_alpha=1.0, fill_alpha=0, outline_color='black')\
        .pl.show(coordinate_systems='global', title="Basic Annotation")

In [ ]:
sdata_cropped.tables["table"]

## CellPose Segmentation

In [ ]:
sopa.make_image_patches(sdata_cropped, patch_width=512, patch_overlap=0, image_key='morphology_focus')
sdata_cropped

In [ ]:
sdata_cropped.pl.render_images('he_image')\
    .pl.render_shapes('image_patches', outline_alpha=1.0, fill_alpha=0, outline_color='black')\
        .pl.show(coordinate_systems='global', title="Patching")

In [ ]:
sopa.utils.get_channel_names(sdata_cropped, image_key='morphology_focus')

In [ ]:
sopa.segmentation.cellpose(sdata_cropped, image_key='morphology_focus', channels=["DAPI"], diameter=30, flow_threshold=2, cellprob_threshold=-6, min_area=400)

In [ ]:
sdata_cropped
# Add a comment for "tab completions"
# Does this work?


In [ ]:
sdata_cropped.pl.render_images('he_image').pl.render_shapes('cellpose_boundaries', fill_alpha=0.0, outline_alpha=0.5, outline_color='black').pl.show(title='cellpose')

In [ ]:
sdata_cropped.pl.render_images('he_image').pl.show(title='he only')

## Baysor Segmentation (Optional)

## ComSeg Segmentation (Optional)

In [ ]:
sdata_cropped.tables['table']

In [ ]:
sopa.make_transcript_patches(
    sdata_cropped,
    patch_width=400,
    patch_overlap=0,
    prior_shapes_key="cellpose_boundaries",
    write_cells_centroids=True,
)

In [ ]:
sdata_cropped

In [ ]:
sdata_cropped.pl.render_images('he_image').pl.render_shapes('transcripts_patches', fill_alpha=0.0, outline_alpha=0.5, outline_color='black').pl.show(title='cellpose')

In [ ]:
sdata_cropped.tables["table"]

In [28]:
config = {'dict_scale': {'x': 1, 'y': 1, 'z': 1}, 'mean_cell_diameter': 15, 'max_cell_radius': 15, 'k_nearest_neighbors': 5, 'norm_vector': False, 'alpha': 0.5, 'allow_disconnected_polygon': True, 'min_rna_per_cell': 10, 'gene_column': 'feature_name'}

In [ ]:
sopa.segmentation.comseg(sdata_cropped, config=config)

In [ ]:
sdata_cropped

In [ ]:
sdata_cropped.pl.render_images('he_image').pl.render_shapes('comseg_boundaries', fill_alpha=0.0, outline_alpha=0.5, outline_color='black').pl.show(title='comseg')

In [ ]:
sdata_cropped.pl.render_images('he_image').pl.render_shapes('cellpose_boundaries', fill_alpha=0.0, outline_alpha=0.5, outline_color='black').pl.show(title='cellpose')

## Aggregating Transcripts to Cells

In [ ]:
sopa.aggregate(sdata_cropped, aggregate_genes=True, aggregate_channels=True, image_key='morphology_focus', shapes_key="cellpose_boundaries", gene_column="feature_name", min_transcripts=10, key_added='cellpose_table')

In [ ]:
sopa.aggregate(sdata_cropped, aggregate_genes=True, aggregate_channels=True, image_key='morphology_focus', shapes_key="comseg_boundaries", gene_column="feature_name", min_transcripts=10, key_added='comseg_table')

In [ ]:
sdata_cropped.tables["cellpose_table"]

In [ ]:
sdata_cropped.tables["comseg_table"]

## Annotating Cells with Tangram (Optional)

In [ ]:
import anndata

adata_reference = anndata.read_h5ad("adata_reference.h5ad")

sopa.utils.tangram_annotate(sdata_cropped, adata_reference, "cell_type")

### Staining-based

- For now, our fluorescence-based annotation is very simple. We provide a dictionary where a channel is associated with a population. Then, each cell is associated with the cell type whose corresponding channel is the brightest (according to a certain Z-score).

## Visualize with Xenium Explorer

In [ ]:
sdata_cropped

In [44]:
del sdata_cropped.tables["table"]

In [ ]:
sopa.io.explorer.write("crop.explorer", sdata_cropped, image_key='he_image')

### Adding cluster assignments

In [ ]:
adata = sdata_cropped.tables["comseg_table"]
adata

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.1)

In [ ]:
sc.pl.umap(adata, color="leiden")


In [ ]:
explorer_path = "crop.explorer"
sopa.io.explorer.write_cell_categories(explorer_path, adata)

## Visualize with Napari

In [ ]:
interactive = Interactive(sdata_cropped)
interactive.run()

## Sample Analysis Report

In [ ]:
sdata_cropped.tables["cellpose_table"]

In [ ]:
sopa.io.write_report("report.html", sdata_cropped)

## Save your zarr file for future analysis

In [ ]:
sdata_cropped.write("sdata_cropped.zarr")